# Analizando datos de Enfermedades Transmitidas por Vector (Dengue) en México para los años 2021 y 2023

### Importando librerías

In [18]:
import sqlite3
import pandas as pd
import numpy as np

### Conexión a la BD

In [37]:
# Se crea la conexión a la BD
connection = sqlite3.connect("dengue.db")

# Comprobar que la base de datos se ha creado correctamente.
print (connection.total_changes)

# Crear el cursor que te va a permitir enviar instrucciones en SQL.
cursor = connection.cursor()

0


Probemos la conexión ejecutando una consulta que nos regrese el número de registros que tiene la tabla:

In [38]:
cursor.execute("SELECT count(*) FROM dengue_301221")
rows = cursor.fetchall()

print(rows)

[(35413,)]


Primero, creamos una vista que contenga todos los registros donde la Entidad de Residencia sea Sonora:

In [ ]:
# Se crea vista

cursor.execute("CREATE VIEW vw_dengue_sonora AS SELECT * FROM dengue_301221 WHERE ENTIDAD_RES = 26")
rows = cursor.fetchall()
print(rows)

Ahora podemos realizar consultas sobre la vista:


In [40]:
df_dengue_sonora = pd.read_sql_query("SELECT * FROM vw_dengue_sonora", connection)
df_dengue_sonora

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,2021-12-13,913179,2,15,26,66,2,2,26,66,...,2,2,2,2,5,1,5,3,26,66
1,2021-12-13,920550,2,50,26,30,2,2,26,30,...,2,2,2,2,5,2,5,1,26,30
2,2021-12-13,910208,1,52,26,18,2,2,26,18,...,2,2,2,1,4,1,5,1,26,18
3,2021-12-13,920833,2,13,26,18,2,2,26,18,...,2,2,2,2,5,2,5,1,26,18
4,2021-12-13,911975,2,38,26,30,2,2,26,30,...,2,2,2,2,5,1,5,3,26,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,2021-12-13,923505,1,52,26,30,2,2,26,30,...,2,2,2,2,5,1,5,3,26,30
433,2021-12-13,923800,2,24,26,3,2,2,26,42,...,2,2,2,2,5,1,5,2,26,3
434,2021-12-13,894915,2,44,26,29,2,2,26,29,...,2,2,2,2,5,1,5,3,26,29
435,2021-12-13,901370,2,25,26,29,2,2,26,29,...,2,2,2,2,5,1,5,3,26,29


Casos confirmados para Dengue y Ch en Sonora:

In [41]:

df_confirmados_sonora = pd.read_sql_query("SELECT * \
                                            FROM vw_dengue_sonora \
                                            WHERE Dictamen = 1 \
                                                OR Dictamen = 2" \
                                        , connection)
df_confirmados_sonora



,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG


### Cerrando la conexión a la BD

In [42]:
connection.commit()

# Cierra la conexión con la base de datos.
connection.close()